In [6]:
pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
import logging
import json
from IPython.display import Pretty, HTML
import urllib3

from pyfhirclient.FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

# Disable the unverified HTTPS request warnings
urllib3.disable_warnings()

In [8]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(
        base_url='https://concept01.ehealthexchange.org:52780/fhirproxy/r4/',
        verify_ssl=False,
        extra_headers={
            'X-DESTINATION': 'CernerHelios',
            'X-POU': 'TREAT',
            # 'Host': 'concept01.ehealthexchange.org:52780'
        })

    fhir_client.auth_type = 'Basic'
    fhir_client.token = 'c3ZjX2VIeEZISVJTYW5kYm94OndpbGxmdWxTdHJvbmdTdGFuZHVyZDc'

    return fhir_client

In [9]:
client = setup_fhir_client()

search_results = client.search('Patient', {
    'family': 'Blackstone',
    'given': 'Veronica'
})
Pretty(json.dumps(search_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "daf3542f-c1ae-4c37-ba9b-9a605b248cfd",
    "type": "searchset",
    "total": 0,
    "link": [
        {
            "relation": "self",
            "url": "https://fhir-ehr.cerner.com/r4/e8a84236-c258-4952-98b7-a6ff8a9c587a/Patient?family=Blackstone&given=Veronica"
        }
    ]
}

There should be exactly one `Patient` matching the search.

In [10]:
patient = search_results['entry'][0]
Pretty(json.dumps(patient, indent=4))

KeyError: 'entry'

In [ ]:
HTML(patient['resource']['text']['div'])

In [ ]:
everything = client.everything('Patient', patient['resource']['id'], count=1000)
Pretty(json.dumps(everything, indent=4))

`MedicationAdministration` should have the medications administered to this Patient.

In [ ]:
medication_administration_results = client.search('MedicationAdministration', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(medication_administration_results, indent=4))

`Observation` should have lab test results, and pregnancy status.

In [ ]:
observation_search_results = client.search('Observation', query_params={
    'patient': patient['resource']['id']
})

Pretty(json.dumps(observation_search_results, indent=4))        

`Encounter` should have hospitalization status.

In [ ]:
encounter_search_results = client.search('Encounter', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(encounter_search_results, indent=4))

In [ ]:
client.close()